In [1]:
import numpy as np
import re

roll_number = re.compile(r'([1,2][0,5,6,7,8,9].\d*)|tezansahu')

class Post(object):
    def __init__(self,author,subject,raw_text):
#         print(author,subject)
        self.author = re.findall(roll_number,author)[0]
        self.subject = re.findall(roll_number,subject)[0]
        self.raw_text = raw_text
        self.text,self.mentions = self.get_text_mentions(raw_text)
    
    def __str__(self):
        return f"{self.author} wrote the following about {self.subject} : \n {self.text}"
 
    def get_text_mentions(self,raw_text):
        # TODO
        return raw_text, None
    def asdict(self):
        return {"Author":self.author,"Subject":self.subject,"Text":self.text}


class Person(object):
    def __init__(self,id,name,degree,dept,hostel,ibs):
        self.id = re.findall(roll_number,id)[0]
        self.name = name
        self.degree = degree
        self.dept = dept
        self.hostel = hostel
        self.ibs = ibs   # TODO see how to put into sql
        self.posts_auth = []
        self.posts_subj = []
        self.posts_tagged = []
    def __str__(self):
        
        ibs_str = ", ".join(list(self.ibs))
        return f"Name: {self.name}, Program: {self.degree}, Dept: {self.dept}, Hostel: {self.hostel}, Clubs Associated with: {ibs_str}"
    
    def compute_post_stats(self):
        self.auth_len = np.mean([len(x.text.split()) for x in self.posts_auth])
        self.subj_len = np.mean([len(x.text.split()) for x in self.posts_subj])
        
    def asdict(self):
        return {"Name":self.name,"id":self.id,"Degree":self.degree,"Department":self.dept,"Hostel":self.hostel,
                "IBs":self.ibs,"Gender":self.gender, "Posts Auth":[x.asdict() for x in self.posts_auth],
                "Posts Subj":[x.asdict() for x in self.posts_subj]
               ,"Posts Tagged":[x.asdict() for x in self.posts_tagged]}

def get_gender(name):
    if name.lstrip().startswith("Ms."):
        print("Girl")
        return "Female"
    else:
        return "Male"
from bs4 import BeautifulSoup as soup

def get_pers(page,url):
    page = soup(page,parser="html.parser")
    personal_info = page.find("ul",class_="list-unstyled puser-details-list").find('li')

    name = personal_info.find('h2').text.split('\n')[0]
    print(name)
    dets = personal_info.find_all('h5')#.text

    try:
        degree,dept,hostel = dets[0].text.split('|')
    except:
        degree,dept,hostel = None,None,None
    try:
        ibs = (dets[1].text.split('|'))
    except:
        ibs = []

    new_pers = Person(url,name,degree,dept,hostel,ibs)

    incoming_edges = page.find("div",id="myTabContent").find_all("div",class_="sub-post-container")
    print(len(incoming_edges))
    for post in incoming_edges:
        parent = post.find_parent('div')
        if "tab-pane" not in parent["class"] or "impression" in parent["id"]:
            # This is done to make sure that only posts are taken in
            continue
        author = post.find("div",class_="user-details").find("a")
        author_id = author['href']
        author_name = author.text  # This is not strictly necessary, keeping it for now
        subject = author.find_next("div",class_="user-details").find("a")
        subject_id = subject['href']
        subject_name = subject.text

        post_text = post.find("p",class_="post-para").text
        new_post = Post(author_id,subject_id,post_text)
        if new_pers.id == new_post.subject:  # Handle the case of two people with the same name
            new_pers.posts_subj.append(new_post)
        elif new_pers.id == new_post.author:
            new_pers.posts_auth.append(new_post)

        else:   # Need to handle mentions separae
            new_pers.posts_tagged.append(new_post)

    new_pers.compute_post_stats()
    return new_pers
# print(new_pers.auth_len,new_pers.subj_len)

In [4]:
# In this cell I try to get the roll numbers of everyone from the xlsx
import pandas as pd
import requests
import json 

df = pd.read_excel('roll_list.xls',header=None)
roll_numbers = df.loc[:,1]
names = df.loc[:,2]
all_people = {}
for idx,roll in (enumerate(roll_numbers)):
#     if idx < 730:
#         continue

    try:
        roll = str(roll).strip()
        url = f"https://yearbook.sarc-iitb.org/student-{roll}/"
#         page = requests.get(url)
        print(roll)
        page = open(f"raw_data_wednesday/{roll}.html","r").read()
#         file = open(f"raw_data_wednesday/{roll}.html","w")
#         file.write(page.text)
#         file.close()
#         pers = get_pers(page.text,url)
        pers = get_pers(page,url)
        pers.gender = get_gender(names[idx])
        all_people[roll] = (pers.asdict())
#         file = open("raw_data.jsonl","a")
#         print(json.dumps(pers.asdict()),file=file)
#         file.close()
    except Exception as e: 
        print(f"Skipping {roll} because {e}")

json.dump(all_people,open("raw_data.json","w"))


170010001
Meena Sourav Ramkishor 
0
170010002
Sachin Vasava 
1
170010003
Piyush Jirwankar [ aka Jeeru ]
5
170010004
Saieesh Jirwankar 
26
170010005
Tarang Jain 
27
170010006
Harshal Kataria 
3
170010007


/home/anshuln/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/anshuln/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Dhruv Jain [ aka DeeJay ]
26
170010008
Gunjan Chitlange 
22
170010009
Rohit Vinodrao Tembhare 
7
170010010
Prajwal Vinodrao Bijwe [ aka Prajwal ]
8
170010011
Atharva Vijay Kanakdande 
7
170010012
Supriya Basappa Kamble 
4
Girl
170010013
Karthik Anantharaman 
4
170010014
Neilabh Banzal 
5
170010015
Trideep Das 
0
170010016
Prathamesh More 
4
170010017
Prince Rudranarayan Sharma 
3
170010018
Rushikesh Hiraman Zoting [ aka Rishi ]
30
170010019
Lahoti Shantanu Shyamkumar 
2
170010020
Karan Suresh Jagdale 
19
170010021
Shinde Kamlesh Dattatray 
10
170010022
Ajit Patwardhan 
18
170010023
Shyam Nitin Thombre 
2
170010024
Vaibhav Takale [ aka Takale ]
34
170010025
Aman Gopalrao Barahate 
0
170010026
Abhishek Meena [ aka Meena ]
4
170010027
Tushar Mishra 
36
170010028
Rajat Gupta 
1
170010029
Sanskriti Bhansali [ aka Sansii ]
67
Girl
170010030
Parag Khandelwal [ aka Paro ]
37
170010031
Lavanya Singh [ aka Lava ]
6
Girl
170010032
Sakshi Dayal 
60
Girl
170010033
Maharshi Pathak 
21
170010034
Yash

VIBHAV BHARGAVA 
40
170040078
Shantanu Chhaparia 
35
170040079
Amey Sharma 
15
170040080
Anshuman Malliwal [ aka Unne ]
23
170040081
Prateek Yadav [ aka Rajaji ]
26
170040082
Aditya Shah 
30
170040083
Siddhant Kurmi 
11
170040084
Poornima Dongre 
32
Girl
170040085
Yash Mishra 
3
170040086
Ankit Singh 
9
170040087
Saurabh Kumar Tripathi 
6
170040088
Shubham Singh 
8
170040089
Prabhat Singh 
5
170040090
Shivam Jaiswal [ aka Jaiswal ]
18
170040091
Matcha Sai Sathwik 
1
170040092
Priyatham Peddinti [ aka Peddinti, Priya ]
5
170040093
Patina Sriharsha [ aka Harsha ]
2
170040094
Shubham Soni [ aka Soni ]
22
170040095
Rohit Chaudhary [ aka AtmaNirbharBharat ]
32
170040096
Mahima Sethi 
28
Girl
170040097
Aniket Mondal 
12
170040098
Tamoghno Pramanik 
28
170040099
Syed Muztaba Ali 
5
170040100
Makineni Bhanu Chandu 
12
170040101
Nikhil Dasari [ aka Nick ]
2
170040102
Savalapurapu Eswar [ aka Binny ]
9
170040103
Lalit Marada 
27
170040104
V A Adithya Upadhyaya [ aka Saavu ]
50
170040105
Shashank

Girl
170100038
Rohan Ramesh Gulve 
2
170100039
Yash Mandhana 
7
170100040
Shaurya Sarna 
39
170100041
Anuroop Nag 
33
170100042
Amartya Mishra 
27
170100043
Prafull Gupta 
50
170100044
Ayush Agrawal [ aka Aayush ]
22
170100045
Rohit Kumar Meena 
0
170100046
Anshul Agarwal 
0
170100047
Saral Mittal 
9
170100048
Anurag Kumar [ aka Anu ]
6
170100049
Naman Agarwal 
10
170100050
Shikhar Agrawal [ aka Shikhar ]
0
170100051
Rishav Arjun 
15
170100052
Harshit Madan [ aka Harshit ]
0
170100053
Tanay Mangal [ aka Chacha ]
17
170100054
Md. Saif Ansari 
27
170100055
Rajat Pitliya 
11
170100056
Bhupendra Singh Kachhawa 
0
170100057
Bhavik Suthar 
14
170100058
Aman Maan 
1
170100059
Deepak Choudhary 
1
170100060
Sandeep Sagar 
0
170100061
Sunil Kumar Jat [ aka Jaat ]
30
170100062
Pankaj Kumar Meena 
0
170100063
Ankur Meshram 
13
170100064
Gayatri Kumbang 
0
Girl
170100065
Anand Iyer 
2
170100066
Puneet Shrivas 
3
170100067
Akshay Choudhary 
0
170100068
Animesh Agrawal 
1
170100069
Abhishek Dawar 
0


Ayush Raj Mehrotra [ aka Ayushraj ]
18
17D070035
Nishant Goel 
4
17D070036
Aditya Khanna 
10
17D070037
Shreyas Goenka 
2
17D070038
Divyansh Ahuja 
8
17D070039
Yagya Mundra 
52
17D070040
Pradeep Kadela [ aka Kadela ]
0
17D070041
Tanmay Goyal 
2
17D070042
Karan Amaliya 
0
17D070043
Jaideep Kharadi 
2
17D070044
Himanshi Mehta 
0
Girl
17D070045
Sonal Kumar 
10
17D070046
Nitish Kumar Das 
0
17D070047
Mohd Safwan 
3
17D070048
Abhijeet Verma 
2
17D070049
Shailee Suryawanshi [ aka Jeni ]
85
Girl
17D070050
Deepanshu Koshta 
0
17D070051
Ritesh Botcha 
5
17D070052
Etcherla Harshavardhan [ aka Harshavardhan ]
6
17D070053
ARYAN LALL [ aka Aryan ]
13
17D070054
Vemula Madhavan 
15
17D070055
Agulla Surya Bharath 
0
17D070056
Tirupati Saketh Chandra [ aka Saketh ]
0
17D070057
Prashant Kurrey 
26
17D070058
J S Mahesh 
0
17D070059
Yash Sharma 
39
17D070060
Malladi Aditya 
1
17D070061
Vanapali Anirudh 
3
17D070062
Jeetika - [ aka Jetzz ]
2
Girl
17D070063
Prachi Goel 
1
Girl
17D070064
Manas Vashistha 
0
17